# Selector Group Chat
`SelectorGroupChat` is a group chat similar to `RoundRobinGroupChat`, but with a model-based next speaker selection mechanism.

In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_core import Image as AGImage
from autogen_agentchat.teams import RoundRobinGroupChat

from pydantic import BaseModel
from PIL import Image
from io import BytesIO
import requests

from IPython.display import Markdown, display
from dotenv import load_dotenv
import os

load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')
model_name = os.getenv("GEMINI_MODEL")

### Model Client

In [3]:
model_client = OpenAIChatCompletionClient(
    model=model_name,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=google_api_key,
    model_info={
        "vision": True,
        "function_calling": True,
        "json_output": True,
        "family": "gpt-4o",
        "structured_output": True
    },
)

### Agents

#### Planning Agent

In [4]:
planning_agent = AssistantAgent(
    name = 'PlanningAgent',
    description= 'An agent for planning tasks,this agent should be the first to engage when given a new task.',
    model_client=model_client,
    system_message='''
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are :
        WebSearchAgent : Searches for information.
        DataAnalystAgent : Performs calculations

    You only plan and delegate tasks - you do not exectue them yourself.

    When assigning tasks, use the below format:
    1. <agent> : <task>

    After all the tasks are completed, summarize the findings and end with "TERMINATE"
    ''',
)

#### Search Web Tool Agent

In [6]:
def search_web_tool(query:str)-> str:
    # Simulate a web search
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web_tool],
    system_message='''
        You are a web search agent.
        Your only tool is search_web_tool - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

#### Data Analyst Agent 

In [7]:
def percentage_change_tool(start:float, end:float) -> float:
    # Calculate percentage change
    if start == 0:
        return 0
    return ((end - start) / start) * 100


data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.

        If you have not seen the data, ask for it.

    ''',
)

## Termination Conditions

A termination condition is a callable that takes a sequence of BaseAgentEvent or BaseChatMessage objects since the last time the condition was called, and returns a StopMessage if the conversation should be terminated, or None otherwise. Once a termination condition has been reached, it must be reset by calling reset() before it can be used again.